In [1]:
import findspark

from pyspark.sql.functions import *


In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
df_train = spark.read.format("csv")\
.option("header", "True")\
.option("inferSchema","True")\
.load("train_ver2.csv")

In [35]:
df_train.show(1000)

+-------------------+---------+------------+---------------+----+---+-------------------+---------+----------+------+-----------+-----------+-------+------+-------------+-------+-------+--------+--------------------+---------------------+------------------+------------------+-----------------+-----------------+----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------+---------------+-----------------+---------------+
|         fecha_dato| ncodpers|ind_empleado|pais_residencia|sexo|age|         fecha_alta|ind_nuevo|antiguedad|indrel|indrel_1mes|tiprel_1mes|indresi|indext|canal_entrada|indfall|tipodom|cod_prov|             nomprov|ind_actividad_cliente|             renta|          segmento|ind_ahor_fin_ult1|ind_a

In [6]:
#Informações sobre dataset
df_train.printSchema()

root
 |-- fecha_dato: timestamp (nullable = true)
 |-- ncodpers: double (nullable = true)
 |-- ind_empleado: string (nullable = true)
 |-- pais_residencia: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- age: string (nullable = true)
 |-- fecha_alta: timestamp (nullable = true)
 |-- ind_nuevo: string (nullable = true)
 |-- antiguedad: string (nullable = true)
 |-- indrel: string (nullable = true)
 |-- ult_fec_cli_1t: timestamp (nullable = true)
 |-- indrel_1mes: string (nullable = true)
 |-- tiprel_1mes: string (nullable = true)
 |-- indresi: string (nullable = true)
 |-- indext: string (nullable = true)
 |-- conyuemp: string (nullable = true)
 |-- canal_entrada: string (nullable = true)
 |-- indfall: string (nullable = true)
 |-- tipodom: string (nullable = true)
 |-- cod_prov: string (nullable = true)
 |-- nomprov: string (nullable = true)
 |-- ind_actividad_cliente: string (nullable = true)
 |-- renta: double (nullable = true)
 |-- segmento: string (nullable = true

In [7]:
#Mudar tipo do dado
df_train = df_train.withColumn(
    colName = 'age', 
    col = regexp_replace('age',',','.').cast('int')
    )

# Inspecionando o resultado
df_train.select('age').printSchema()

root
 |-- age: integer (nullable = true)



In [8]:
df_train = df_train.withColumn(
    colName = 'antiguedad', 
    col = regexp_replace('antiguedad',',','.').cast('int')
    )

# Inspecionando o resultado
df_train.select('antiguedad').printSchema()

root
 |-- antiguedad: integer (nullable = true)



In [9]:
df_train = df_train.withColumn(
    colName = 'indrel_1mes', 
    col = regexp_replace('indrel_1mes',',','.').cast('int')
    )

# Inspecionando o resultado
df_train.select('indrel_1mes').printSchema()

root
 |-- indrel_1mes: integer (nullable = true)



In [10]:
#Excluir colunas com maioria nulos
df_train=df_train.drop('conyuemp')
df_train=df_train.drop('ult_fec_cli_1t')

In [11]:
#Tratar nulos de segmento
df_train = df_train.na.fill("02 - PARTICULARES",subset=["segmento"])

In [12]:
df_train.groupBy("segmento").count().sort(desc("count")).show()

+------------------+-------+
|          segmento|  count|
+------------------+-------+
| 02 - PARTICULARES|8149588|
|03 - UNIVERSITARIO|4935579|
|          01 - TOP| 562142|
+------------------+-------+



In [32]:
#Media da idade
mean_ = df_train.agg({'age': 'avg'}).show()

#tratar nulos
df_train = df_train.na.fill(40.18146491355274,subset=["age"])

+-----------------+
|         avg(age)|
+-----------------+
|40.18146491355274|
+-----------------+



In [33]:
df_train.groupBy("age").count().sort(desc("count")).show()

+---+------+
|age| count|
+---+------+
| 23|779884|
| 22|736314|
| 24|734785|
| 21|675988|
| 25|472016|
| 20|422867|
| 26|347778|
| 43|324303|
| 44|322955|
| 42|319713|
| 40|315488|
| 45|314771|
| 41|309051|
| 46|299365|
| 47|286505|
| 27|281981|
| 48|271576|
| 39|260548|
| 49|250484|
| 28|240192|
+---+------+
only showing top 20 rows



In [46]:
df_train = df_train.na.fill("V",subset=["sexo"])
df_train.groupBy("sexo").count().sort(desc("count")).show()

+----+-------+
|sexo|  count|
+----+-------+
|   V|7452056|
|   H|6195253|
+----+-------+

